# Impedance Calibration 

In [1]:
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
from stochopy.optimize import minimize
from stochastic_optimization import objective_function

In [2]:
fp = Path.home() / 'Documents/BikewaySimData/Projects/gdot'

In [3]:
with (fp / 'chosen.pkl').open('rb') as fh:
    df_edges,pseudo_df,pseudo_G = pickle.load(fh)

In [4]:
#add teh geometry 
fp = Path.home() / "Documents/BikewaySimData/Projects/gdot"
edges = gpd.read_file(fp/'networks/elevation_added.gpkg',layer="links")
edges.to_crs('epsg:2240',inplace=True)
df_edges = df_edges.merge(edges[['linkid','geometry']],on=['linkid'])
df_edges = gpd.GeoDataFrame(df_edges,geometry='geometry',crs=edges.crs)
df_edges = df_edges.loc[:,~df_edges.columns.duplicated()].copy()
df_edges.reset_index(drop=True,inplace=True)

In [5]:
nodes = gpd.read_file(Path.home()/'Documents/BikewaySimData/projects/gdot/networks/reconciled.gpkg',layer="nodes")[['N','geometry']]
nodes = nodes[nodes['N'].isin(set(df_edges['source'].append(df_edges['target']).tolist()))]

In [6]:
with (Path('D:/matched_traces')/'ready4calibration.pkl').open('rb') as fh:
    matched_traces = pickle.load(fh)

In [7]:
pseudo_df['source_linkid']

0              0
1              1
2              2
3              0
4              1
           ...  
958299    169221
958300    169214
958301    169216
958302    169221
958303    169248
Name: source_linkid, Length: 958304, dtype: int64

In [8]:
df_edges['length_ft'] = df_edges.length

#drop multi-edges
print(df_edges.shape[0])
min_links = df_edges.groupby(['source','target'])['length_ft'].idxmin()
df_edges = df_edges.loc[min_links]
print(df_edges.shape[0])

338486
332517


In [9]:

print(pseudo_df.shape[0])
mask = pseudo_df['source_linkid'].isin(df_edges['linkid']) & pseudo_df['target_linkid'].isin(df_edges['linkid'])
pseudo_df = pseudo_df[mask]
print(pseudo_df.shape[0])

958304
932238


In [10]:
df_edges['high_traffic_stress'] = df_edges['highway'] == 'primary'
# df_edges['bike_facility_type'].value_counts()

In [11]:
# df_edges['high_traffic_stress'] = df_edges['bike_facility_type'].isna() & (df_edges['highway'].map(levels) > 4 | df_edges['speed limit'] > 30)

In [12]:
pseudo_df.columns

Index(['source_A', 'source_B', 'target_A', 'target_B', 'source_reverse_link',
       'source_azimuth', 'source_linkid', 'target_reverse_link',
       'target_azimuth', 'target_linkid', 'azimuth_change', 'turn_type',
       'source', 'target', 'signalized', 'unsignalized'],
      dtype='object')

In [13]:
pseudo_df['left'] = pseudo_df['turn_type'] == 'left'

In [14]:
#have position of beta next to name of variable
beta_links = {
    0 : 'ascent_grade',
    1 : 'high_traffic_stress',
}

beta_turns = {
    2 : 'signalized',
    3 : 'unsignalized',
    4 : 'left',
}

'''
Currently works with binary and numeric variables. Categoricals will have to be
cast into a different format for now.

Link impedance is weighted by the length of the link, turns are just the impedance associated
'''

#customize this function to change impedance formula
#TODO streamline process of trying out new impedance functions
def link_impedance_function(betas,beta_links,links):
    #prevent mutating the original links gdf
    links = links.copy()
    
    links['attr_multiplier'] = 0

    for key, item in beta_links.items():
        links['attr_multiplier'] = links['attr_multiplier'] + (betas[key] * links[item])

    links['link_cost'] = links['length_ft'] * (1 + links['attr_multiplier'])
    
    return links

def turn_impedance_function(betas,beta_turns,pseudo_df):
    #use beta coefficient to calculate turn cost
    base_turn_cost = 30 # from Lowry et al 2016 DOI: http://dx.doi.org/10.1016/j.tra.2016.02.003
    # turn_costs = {
    #     'left': betas[1] * base_turn_cost,
    #     'right': betas[1] * base_turn_cost,
    #     'straight': betas[1] * base_turn_cost
    # }
    #pseudo_df['turn_cost'] = pseudo_df['turn_type'].map(turn_costs)

    pseudo_df = pseudo_df.copy()

    pseudo_df['turn_cost'] = 0

    for key, item in beta_turns.items():
        pseudo_df['turn_cost'] = pseudo_df['turn_cost'] + (betas[key] * pseudo_df[item])

    return pseudo_df

In [15]:
betas = [2,3,4,5,6]
#link_impedance_function(betas,beta_links,df_edges)
#(turn_impedance_function(betas,beta_turns,pseudo_df)['turn_cost'] > 0).sum()

In [16]:
kwargs = {
    'beta_links': beta_links,
    'beta_turns': beta_turns,
    'links': df_edges,
    'pseudo_links': pseudo_df,
    'pseudo_G': pseudo_G,
    'matched_traces': matched_traces,
    'link_impedance_function': link_impedance_function,
    'turn_impedance_function': turn_impedance_function,
    'exact': False,
    'follow_up': False
}
args = tuple(v for k, v in kwargs.items())
len(args)

10

In [17]:
bounds = [[0, 5] for _ in range(0, 5)]
bounds

[[0, 5], [0, 5], [0, 5], [0, 5], [0, 5]]

In [21]:
import stochastic_optimization
from importlib import reload
reload(stochastic_optimization)

start = time.time()
# args = (df_edges,pseudo_df,pseudo_G,matched_traces,False)
x = minimize(stochastic_optimization.objective_function, bounds, args=args, method='pso', options={'maxiter':5})
end = time.time()
print(f'Took {(end-start)/60/60} hours')
#results[segment_filepath] = (x.x,x.fun)

setting link costs
setting turn costs
finding lowest cost
updating edge weights
Shortest path routing with coefficients: [4.09864388 3.0806663  0.02682683 1.05290064 0.12491518]
calculating objective function
Median overlap percent is: 45.8 %
setting link costs
setting turn costs
finding lowest cost
updating edge weights
Shortest path routing with coefficients: [4.52902242 0.51265154 4.01485346 3.61762247 4.61112909]
calculating objective function
Median overlap percent is: 45.8 %
setting link costs
setting turn costs
finding lowest cost
updating edge weights
Shortest path routing with coefficients: [0.16432387 1.11683649 2.74403345 2.74443099 1.70562879]
calculating objective function
Median overlap percent is: 44.9 %
setting link costs
setting turn costs
finding lowest cost
updating edge weights
Shortest path routing with coefficients: [1.71109052 4.63202104 1.11083446 2.0828983  3.56304219]
calculating objective function
Median overlap percent is: 60.0 %
setting link costs
setting t

In [19]:
pseudo_df

,source_A,source_B,target_A,target_B,source_reverse_link,source_azimuth,source_linkid,target_reverse_link,target_azimuth,target_linkid,azimuth_change,turn_type,source,target,signalized,unsignalized,left,source_link_cost,target_link_cost
0,67358019,67358015,67358015,67358019,False,358.5,0,True,178.5,0,180.0,uturn,"(67358019, 67358015)","(67358015, 67358019)",False,False,False,7978.874482,7978.874482
1,783782570,67358015,67358015,67358019,False,270.4,1,True,178.5,0,268.1,left,"(783782570, 67358015)","(67358015, 67358019)",False,False,True,467.459370,7978.874482
2,67377092,67358015,67358015,67358019,False,91.4,2,True,178.5,0,87.1,right,"(67377092, 67358015)","(67358015, 67358019)",False,False,False,7143.501729,7978.874482
3,67358019,67358015,67358015,783782570,False,358.5,0,True,90.4,1,91.9,right,"(67358019, 67358015)","(67358015, 783782570)",False,False,False,7978.874482,467.459370
4,783782570,67358015,67358015,783782570,False,270.4,1,True,90.4,1,180.0,uturn,"(783782570, 67358015)","(67358015, 783782570)",False,False,False,467.459370,467.459370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958299,11700971622,11700971616,11700971616,11700971552,False,351.8,169221,False,7.3,169216,15.5,straight,"(11700971622, 11700971616)","(11700971616, 11700971552)",False,False,False,2240.589685,5724.099709
958300,11700971536,11700971616,11700971616,11700971622,False,165.0,169214,True,171.8,169221,6.8,straight,"(11700971536, 11700971616)","(11700971616, 11700971622)",False,False,False,4936.919925,2240.589685
958301,11700971552,11700971616,11700971616,11700971622,True,187.3,169216,True,171.8,169221,344.5,straight,"(11700971552, 11700971616)","(11700971616, 11700971622)",False,False,False,5724.099709,2240.589685
958302,11700971622,11700971616,11700971616,11700971622,False,351.8,169221,True,171.8,169221,180.0,uturn,"(11700971622, 11700971616)","(11700971616, 11700971622)",False,False,False,2240.589685,2240.589685


## Impedance Function 2
- Link Specific:
    - Average Grade (%grade)
    - Vehicle Seperation from OSM/ARC Inventory (1 = None, 2 = Bike Lane, 3 = MUP/Curb protected bike lanes)
    - Number of lanes from HERE ()
- Turn Specific
    - Unsignalized left/straight across roads with higher than tertiary classification (0 or 1)
    - Signalized intersection left/straight (0 or 1)

## Applying Link Costs
---
Dict keys must correspond to column names in links GeoDataFrame. Multiple dicts can be passed to pseudo_df the impacts of changing impedances. The links cost function is of this format:
$$ C_e = \frac{l_e*60^2}{s*5280} * (1-\sum \beta_i x_{i,e}) $$

where:
- $e$ is an edge/link in network graph $G$ with V vertices/nodes and E edges/links
- $l_e$ is the length of the link in feet
- $\beta$ is the impedance coefficient for attribute $i$
- $X_{i,e}$ is the value of attribute $i$ for link $e$
- $s$ is the assumed average speed of the cyclist in mph

Notes:
- Negative attributes **decrease** impedance  
- Positive attributes **increase** impedance
- **Negative link costs are not allowed**
- Time to traverse a link has already been calculated in the prepare_network function

In [ ]:
# #%% prepare link dataframe
# links['bike'] = links['bl'] + links['pbl'] + links['mu']
# links['bike'] = links['bike'] >= 1

# cost_columns = ['linkid','bike','length_ft']#,'up-grade','down-grade','length_ft']
# df_edges = df_edges.merge(links[cost_columns],on='linkid')

# # df_edges['grade'] = np.nan
# # df_edges.loc[df_edges['reverse_link'],'grade'] = df_edges['down-grade']
# # df_edges.loc[~df_edges['reverse_link'],'grade'] = df_edges['up-grade']
# # #ignore downs
# # df_edges.loc[df_edges['grade']<0,'grade'] = 0
# # df_edges.drop(columns=['up-grade','down-grade','bearing'],inplace=True)

In [ ]:
# #fix set
# import ast
# matched_traces['linkids'] = matched_traces['linkids'].apply(lambda x: eval(x))

In [ ]:
#drop loops
matched_traces = matched_traces.loc[matched_traces['start']!=matched_traces['end']]

In [ ]:
# with (fp / 'impedance_calibration.pkl').open('rb') as fh:
#     (df_edges,pseudo_df,pseudo_G) = pickle.load(fh)
args = (df_edges,pseudo_df,pseudo_G,matched_traces,False)

In [ ]:
import stochastic_optimization
from importlib import reload
reload(stochastic_optimization)

In [ ]:
# source = 68294161
# target = 2400730083

# pseudo_G, virtual_edges = modeling_turns.add_virtual_links(pseudo_df,pseudo_G,source,[target])   

# virtual_edges

# pseudo_G.out_edges(target)
# #pseudo_G.in_edges((5416154182, 2400730083))

# list(pseudo_G.in_edges(target))[0]

# test = nx.ego_graph(pseudo_G,source,4)
# test.edges()

# import networkx as nx
# test_target = (5318092552,5416166514)

# length, node_list = nx.single_source_dijkstra(pseudo_G,source,test_target,weight='weight')
# node_list

# pseudo_G = modeling_turns.remove_virtual_edges(pseudo_G,virtual_edges)

# import stochastic_optimization
# from importlib import reload
# reload(stochastic_optimization)
# reload(modeling_turns)

# betas = [1.14593853, 0.60739776]
# val, merged = stochastic_optimization.objective_function(betas,*args)

# merged[1].set_geometry('geometry_modeled').set_crs('epsg:2240').explore()

Need to re-create routes using the coefficients so we can do vizualization

In [ ]:
import stochastic_optimization
from importlib import reload
reload(stochastic_optimization)

betas = np.array([0.09231109, 2.35131751])
args = (df_edges,pseudo_df,pseudo_G,matched_traces,False,True)
test = stochastic_optimization.objective_function(betas,*args)

# Visualization

In [ ]:
test.columns

test['percent_detour'] = (((test['length_ft']-test['shortest_length_ft'])/test['shortest_length_ft'])*100).round(1)


In [ ]:
import pandas as pd
trip_and_user = pd.read_pickle(export_fp/'trip_and_user.pkl')

test_merge = test.merge(trip_and_user,on='tripid')

In [ ]:
tripid = test.loc[test['overlap']<0.2,'tripid'].sample(1).item()
tripid

In [ ]:
row['starttime']

In [ ]:
import folium
import geopandas as gpd
from folium.plugins import MarkerCluster, PolyLineTextPath
from folium.map import FeatureGroup

def visualize(test_merge,tripid):


     gdf = test_merge.copy()

     gdf.set_geometry("geometry",inplace=True)
     gdf.set_crs("epsg:2240",inplace=True)

     # Your GeoDataFrames
     chosen_path = gdf.loc[gdf['tripid']==tripid,['tripid','geometry']]
     shortest_path = gdf.loc[gdf['tripid']==tripid,['tripid','shortest_geo']].set_geometry('shortest_geo').set_crs(gdf.crs)
     intersection = gdf.loc[gdf['tripid']==tripid,['tripid','shortest_intersect_geo']].set_geometry('shortest_intersect_geo').set_crs(gdf.crs)
     modeled_path = gdf.loc[gdf['tripid']==tripid,['tripid','geometry_modeled']].set_geometry('geometry_modeled').set_crs(gdf.crs)

     #start point
     start_N = gdf.loc[gdf['tripid']==tripid,'start'].item()
     start_pt = nodes.to_crs('epsg:4326').loc[nodes['N']==start_N,'geometry'].item()

     #end point
     end_N = gdf.loc[gdf['tripid']==tripid,'end'].item()
     end_pt = nodes.to_crs('epsg:4326').loc[nodes['N']==end_N,'geometry'].item()

     # reproject
     x_mean = chosen_path.to_crs(epsg='4326').geometry.item().centroid.x
     y_mean = chosen_path.to_crs(epsg='4326').geometry.item().centroid.y

     # Create a Folium map centered around the mean of the GPS points
     center = [y_mean,x_mean-.04]
     mymap = folium.Map(location=center, zoom_start=13)

     # Convert GeoDataFrames to GeoJSON
     chosen_path_geojson = chosen_path.to_crs(epsg='4326').to_json()
     shortest_path_geojson = shortest_path.to_crs(epsg='4326').to_json()
     intersection_geojson = intersection.to_crs(epsg='4326').to_json()
     modeled_path_geojson = modeled_path.to_crs(epsg='4326').to_json()

     # Create FeatureGroups for each GeoDataFrame
     chosen_path_fg = FeatureGroup(name='Chosen Path')
     shortest_path_fg = FeatureGroup(name='Shortest Path')
     intersection_fg = FeatureGroup(name='Buffer Intersection',show=False)
     modeled_path_fg = FeatureGroup(name='Modeled Path')

     # Add GeoJSON data to FeatureGroups
     folium.GeoJson(chosen_path_geojson, name='Chosen Path',
                    style_function=lambda x: {'color': '#fc8d62', 'weight': 12}).add_to(chosen_path_fg)
     folium.GeoJson(shortest_path_geojson, name='Shortest Path',
                    style_function=lambda x: {'color': '#66c2a5', 'weight': 8}).add_to(shortest_path_fg)
     folium.GeoJson(intersection_geojson, name='Buffer Intersection',
                    style_function=lambda x: {'color':"gray",'fillColor':"#ffff00","fillOpacity": 0.75}).add_to(intersection_fg)
     folium.GeoJson(modeled_path_geojson, name='Modeled Path',
                    style_function=lambda x: {'color': '#8da0cb','weight': 8}).add_to(modeled_path_fg)

     # Add FeatureGroups to the map
     chosen_path_fg.add_to(mymap)
     shortest_path_fg.add_to(mymap)
     intersection_fg.add_to(mymap)
     modeled_path_fg.add_to(mymap)

     # Add start and end points with play and stop buttons
     start_icon = folium.Icon(color='green',icon='play',prefix='fa')
     end_icon = folium.Icon(color='red',icon='stop',prefix='fa')
     folium.Marker(location=[start_pt.y, start_pt.x],icon=start_icon).add_to(mymap)
     folium.Marker(location=[end_pt.y, end_pt.x],icon=end_icon).add_to(mymap)

     # Add layer control to toggle layers on/off
     folium.LayerControl().add_to(mymap)

     #retrive overlap
     # exact_overlap = gdf.loc[gdf['tripid']==tripid,'shortest_exact_overlap_prop'].item()
     # buffer_overlap = gdf.loc[gdf['tripid']==tripid,'shortest_buffer_overlap'].item()
     row = gdf.loc[gdf['tripid']==tripid].squeeze()

     # Add legend with statistics
     #TODO what happened to duration
     legend_html = f'''
          <div style="position: fixed; 
                    bottom: 5px; left: 5px; width: 300px; height: 400px; 
                    border:2px solid grey; z-index:9999; font-size:14px;
                    background-color: white;
                    opacity: 0.9;">
          &nbsp; <b>Trip ID: {tripid}, User ID: {row['userid']}</b> <br>
          &nbsp; <b> Date: {row['starttime']} </b> <br>
          &nbsp; Start Point &nbsp; <i class="fa fa-play" style="color:green"></i>,
          End Point &nbsp; <i class="fa fa-stop" style="color:red"></i> <br>
          
          &nbsp; Chosen Path &nbsp; <div style="width: 20px; height: 5px; background-color: #fc8d62; display: inline-block;"></div> <br>
          &nbsp; Shortest Path &nbsp; <div style="width: 20px; height: 5px; background-color: #66c2a5; display: inline-block;"></div> <br>
          &nbsp; Modeled Path &nbsp; <div style="width: 20px; height: 5px; background-color: #8da0cb; display: inline-block;"></div> <br>
          &nbsp; Buffer Overlap &nbsp; <div style="width: 20px; height: 10px; background-color: #ffff00; display: inline-block;"></div> <br>

          &nbsp; Percent Detour: {row['percent_detour']:.0f}% <br>
          &nbsp; Shortest Path Overlap: {row['shortest_buffer_overlap']*100:.0f}% <br>
          &nbsp; Modeled Path Overlap: {row['overlap']*100:.0f}% <br>
          &nbsp; Trip Type: {row['trip_type']} <br>
          &nbsp; Length (mi): {row['length_ft']/5280:.0f} <br>
          &nbsp; Age: {row['age']} <br>
          &nbsp; Gender: {row['gender']} <br>
          &nbsp; Income: {row['income']} <br>
          &nbsp; Ethnicity: {row['ethnicity']} <br>
          &nbsp; Cycling Frequency: {row['cyclingfreq']} <br>
          &nbsp; Rider History: {row['rider_history']} <br>
          &nbsp; Rider Type: {row['rider_type']} <br><br>

          </div>
          '''
     mymap.get_root().html.add_child(folium.Element(legend_html))

     # Save the map to an HTML file or display it in a Jupyter notebook
     #mymap.save('map.html')
     # mymap.save('/path/to/save/map.html')  # Use an absolute path if needed
     return mymap  # Uncomment if you are using Jupyter notebook

     #TODO add in the legend with trip info and then we're golden


In [ ]:
test_merge.tripid

In [ ]:
#tripid = 891#30000
tripid = 7257#9806#891
mymap = visualize(test_merge,tripid)
mymap